In [ ]:
import pandas as pd
import numpy as np
import random
from math import ceil, sqrt
from shapely.geometry import Point, Polygon
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()

def svgCloudEdge(x0, y0, x1, y1, h=20, r=14, rd=10, _stroke_='#ff0000', _fill_='#ff0000'):    
    vx, vy   = x1 - x0, y1 - y0
    l        = sqrt(vx * vx + vy * vy)
    vx, vy   = vx / l, vy / l
    vxp, vyp = vy, -vx
    _svg_ = f'<path d="m {x0} {y0} l {vxp*h} {vyp*h} l {vx*l} {vy*l} l {-vxp*h} {-vyp*h} z" stroke="{_stroke_}" fill="{_fill_}" />'

    circles = ceil(l / r)
    for i in range(int(circles)+1):
        x, y = x0 + vxp*h + vx*l*i/circles, y0 + vyp*h + vy*l*i/circles
        rr, xr, yr = r + rd*(random.random()-0.5), x + rd*(random.random()-0.5), y + rd*(random.random()-0.5)
        _svg_ += f'<circle cx="{xr}" cy="{yr}" r="{rr}" stroke="{_stroke_}" fill="{_fill_}" />'

    rr     = h/2
    xr, yr = x0 + vxp*rr, y0 + vyp*rr
    _svg_ += f'<circle cx="{xr}" cy="{yr}" r="{rr}" stroke="{_stroke_}" fill="{_fill_}" />'

    rr     = h/2
    xr, yr = x0 + vxp*rr + l*vx, y0 + vyp*rr + l*vy
    _svg_ += f'<circle cx="{xr}" cy="{yr}" r="{rr}" stroke="{_stroke_}" fill="{_fill_}" />'
    return _svg_


In [ ]:
_svg_ =  '<svg width="300" height="300">'
_svg_ += '<rect x="0" y="0" width="300" height="300" fill="#ffffff"/>'
_svg_ += svgCloudEdge(100,  50, 200,  50)
_svg_ += svgCloudEdge(200,  50, 200, 150)
_svg_ += svgCloudEdge(200, 150, 100, 150)
_svg_ += svgCloudEdge(100, 150, 100,  50)
_svg_ += '</svg>'
rt.displaySVG(_svg_)

In [ ]:
import math

def extrudePolygon(core, r): # Only works on convex shapes...
    pts     = core.exterior.coords
    extrusion_pts = []
    for i in range(0, len(pts)-1):
        x0,y0,x1,y1 = pts[i][0], pts[i][1], pts[i+1][0], pts[i+1][1]
        vx, vy      = x1-x0, y1-y0
        l           = math.sqrt(vx*vx + vy*vy)
        if l < 0.001:
            l = 1.0
        vxn, vyn     = vx/l, vy/l
        vxn_p, vyn_p = -vyn, vxn
        extrusion_pts.append((x0+vxn_p*r, y0+vyn_p*r))
        extrusion_pts.append((x1+vxn_p*r, y1+vyn_p*r))
    extrusion = Polygon(extrusion_pts)
    return extrusion

def cloudCircleEdges(core, r):
    circle_tuples = []
    pts     = core.exterior.coords
    for i in range(0, len(pts)-1):
        x0,y0,x1,y1 = pts[i][0], pts[i][1], pts[i+1][0], pts[i+1][1]
        vx, vy      = x1-x0, y1-y0
        l           = math.sqrt(vx*vx + vy*vy)
        if l < 0.001:
            l = 1.0
        vxn, vyn     = vx/l, vy/l
        vxn_p, vyn_p = -vyn, vxn

        t = random.random() * (r/2)
        while t < l:
            from_edge = random.random() * r
            radius    = random.random() * r/2 + r/2
            circle_tuples.append((x0+vxn*t+vxn_p*from_edge, y0+vyn*t+vyn_p*from_edge, radius))
            t += random.random() * (r/2)

    return circle_tuples

def svgCloud(core, r=50):
    svg = ''
    svg += f'<path d="{rt.shapelyPolygonToSVGPathDescription(core)}" fill="none" stroke="#000000"/>'
    svg += f'<path d="{rt.shapelyPolygonToSVGPathDescription(extrudePolygon(core,r))}" fill="none" stroke="#ff0000"/>'
    _tuples_ = cloudCircleEdges(core, r)
    for _t_ in _tuples_:
        svg += f'<circle cx="{_t_[0]}" cy="{_t_[1]}" r="{_t_[2]}" fill="none" stroke="#000000" />'
    return svg

x,y,w,h = 150,150,200,200
my_rectangle = Polygon([(x,y), (x,y+h), (x+w,y+h), (x+w,y)])
rt.displaySVG('<svg x="0" y="0" width="500" height="500"><rect x="0" y="0" width="500" height="500" fill="#ffffff"/>' + svgCloud(my_rectangle) + '</svg>')

In [ ]:
my_rectangle